# Pick Data Analysis

Did not use Time Series analysis because next order pick time is not dependent on previous order pick time


## Background/Setup



## Goal

## Findings Summary

## Model Results Summary

The MAE (median absolute error) of the baseline (using just the mean pick seconds to predict) is 95 seconds     

Using 2 features only: total lines per order (a measure of order complexity) and if the order is picked in the last hour of the day:
- the 2 degree polynomial model MAE is 33 seconds on both in sample and out of sample data
- this is a 65% improvement in prediction accuracy using the model instead of the mean to predict pick time

Using only 1 feature: total lines produced nearly identical results (with .01 of previous results) I conclude that the is_hr_18 feature is not needed for modeling.      

Prediction accuracy of pick seconds per order can be improved by 65% just using the total lines on the order as a feature in a 2 degree polynomial feature model.

# Wrangle

## Environment setup

In [1]:
import acquire
import prepare
import wrangle_pick
import summarize
import explore
import model

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import math
from datetime import datetime

from sklearn.metrics import mean_squared_error, median_absolute_error, explained_variance_score
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import f_regression

from pandas.plotting import register_matplotlib_converters

import warnings
warnings.filterwarnings("ignore")

# This is to make sure matplotlib doesn't throw the following error:
# The next line fixes "TypeError: float() argument must be a string or a number, not 'Timestamp' matplotlib"
pd.plotting.register_matplotlib_converters()

explore.set_plotting_defaults()

In [2]:
df = pd.read_csv('pickdf.csv', index_col=0)

In [5]:
df.columns

Index(['PH_PICKEDB', 'PH_PICKSTA', 'PH_PICKEND', 'PH_TOTALLI', 'PH_TOTALBO'], dtype='object')

In [ ]:
train, validate, test= wrangle_pick.wrangle_pick_data()
train.shape, validate.shape, test.shape

In [ ]:
train.head()

In [ ]:
# found that Xy function was changing original train, validate, test
# try creating copies first so that original are uneffected
X_train = train.copy()
X_validate = validate.copy()
X_test = test.copy()

In [ ]:
# exploration of train shows 2% of orders are more than 1 box, dropping these to reduce noise for mvp
X_train_exp, X_train, X_validate, X_test, y_train, y_validate, y_test = wrangle_pick.createXy(X_train, X_validate, X_test)

X_train_exp.shape, X_train.shape, X_validate.shape, X_test.shape, y_train.shape, y_validate.shape, y_test.shape

In [ ]:
train.shape, X_train_exp.shape, X_train.shape

In [ ]:
# create df for explore where orders with more than 1 box are included
exp = train.copy()

In [ ]:
exp.info()

In [ ]:
# create df for time series exploration includes orders with more than 1 box
ts = train.copy()

# Non-TS index Explore

## Prep explore dataframe

In [ ]:
exp1 = exp[['total_lines', 'total_boxes', 'pick_seconds', 'operator', 'hour', 'day', 'day_name', 
            'week', 'month', 'year', 'sec_per_box', 'lines_per_box', 'sec_per_line' ]]

In [ ]:
exp1.head()

In [ ]:
exp1.info()

In [ ]:
# export this dataframe for exploring in Tableau
exp1.to_csv('exp1.csv')

In [ ]:
exp1 = pd.read_csv('exp1.csv', index_col=0)
exp1.head()

### check for observations where pick_seconds is 0

- none where seconds is 0, 1, or 2 only 3 records where time is 4 seconds

Checking to make sure orders have a minimum time greater than 2 seconds.

In [ ]:
exp1[exp1.pick_seconds==3]

## create visualizations

**DEFINITIONS**     
- each observation is 1 order to fulfill
- the number of lines is the number of (unique?) items to put in the box
- each order is a minimum of 1 box, though more boxes may be needed
    - check if lines are higher on multiple box orders
    - compare mean pick times for multiple box orders

In [ ]:
exp1.head()

In [ ]:
exp1.groupby(['total_boxes']).total_lines.agg(['mean', 'median', 'count'])

In [ ]:
# what is the avg lines for orders with more than 1 box?
avg2plusbox = exp1[exp1.total_boxes >1].total_lines.agg(['mean', 'count'])
# what is the avg line for orders with 1 box?
avg1box = exp1[exp1.total_boxes == 1].total_lines.agg(['mean', 'count'])
# what is the population average?
popavg = exp1.total_lines.agg(['mean', 'count'])
print('average total_lines=', popavg)
print('average lines for orders with 1 box=', avg1box)
print('average lines for orders with 2 or more boxes=', avg2plusbox)

- there are, relatively, very few orders with more than 1 box (only 2K out of 96K)
- might want to drop observations with more than 1 box for MVP model to reduce noise
- due to the imbalance of data the overall average and 1 box average are nearly the same

### Hypothesis Test 1

Is the variance between the average number of lines for the orders with more than 1 box significantly different from the population average?

Ho: The difference is not significant     
Ha: There is a significant difference between the average number of lines      
alpha = .05 (meaning there 95% confidence variation is not due to random chance)     

The p is less than alpha so the null hypothesis (Ho) is rejected

In [ ]:
# use one sample t-test
over1box = exp1[exp1.total_boxes >1].total_lines
popmean = exp1.total_lines.mean()
t, p = explore.ttest_1samp(over1box, popmean)

In [ ]:
exp1.groupby(['total_boxes']).pick_seconds.agg(['mean', 'median', 'count'])

In [ ]:
# what is the avg pick time for orders with more than 1 box?
pickavg2plusbox = exp1[exp1.total_boxes >1].pick_seconds.agg(['mean', 'median', 'count'])
# what is the avg pick time for orders with 1 box?
pickavg1box = exp1[exp1.total_boxes == 1].pick_seconds.agg(['mean', 'median', 'count'])
# what is the average pick time?
pickpopavg = exp1.pick_seconds.agg(['mean', 'median', 'count'])
print('average total_lines=')
print(pickpopavg)
print('average lines for orders with 1 box=')
print(pickavg1box)
print('average lines for orders with 2 or more boxes=')
print(pickavg2plusbox)

### Hypothesis Test 2

Is the variance between the average pick seconds for the orders with more than 1 box significantly different from the population average?

Ho: The difference is not significant     
Ha: There is a significant difference between the average pick seconds for orders with more than 1 box      
alpha = .05 (meaning there 95% confidence variation is not due to random chance)     

The p is less than alpha so the null hypothesis (Ho) is rejected

In [ ]:
# use one sample t-test
pover1box = exp1[exp1.total_boxes >1].pick_seconds
ppopmean = exp1.pick_seconds.mean()
t, p = explore.ttest_1samp(pover1box, ppopmean)

## Explore variation in pick seconds

- each row is an order, so the pick seconds is per order
- maximum outlier is 8000 seconds, or about 2.2 hours
- plot have y axis limit set to 2000 or about 30 min to reduce the visual impact of the outliers

In [ ]:
# For Tableau
exp1.to_csv('explore_all_boxes.csv')
X_train_exp.to_csv('exp_only_1box.csv')

### does the pick seconds vary by hour of day?
- operating hours start at 7a and end at 6p
- lots of outliers
- not much variation until last hour of day
- pattern is the same with only 1 box orders

In [ ]:
sns.boxplot(data=exp1, y='pick_seconds', x='hour')
plt.title('pick seconds by hour')
plt.ylim(0, 2000)
plt.show()
# note: y axis is limited to 2000 seconds, or about 30 min to reduce visual impact of outliers

In [ ]:
sns.boxplot(data=X_train_exp, y='pick_seconds', x='hour')
plt.title('pick seconds by hour with only 1 box orders')
plt.ylim(0, 500)
plt.show()
# note: y axis is limited to 2000 seconds, or about 30 min to reduce visual impact of outliers

### does the pick seconds vary by day of week?
- not much on average, but there seem to be a lot of outliers?
- pattern is the same with only 1 box orders

In [ ]:
order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
sns.boxplot(data=exp1, y='pick_seconds', x='day_name', order=order)
plt.title('pick seconds by day of week')
plt.ylim(0, 2000)
plt.show()
# note: y axis is limited to 2000 seconds, or about 30 min to reduce visual impact of outliers

In [ ]:
order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
sns.boxplot(data=X_train_exp, y='pick_seconds', x='day_name', order=order)
plt.title('pick seconds by day of week only orders with 1 box')
plt.ylim(0, 2000)
plt.show()
# note: y axis is limited to 2000 seconds, or about 30 min to reduce visual impact of outliers

### does the pick seconds vary by day of month?
- no real pattern of variation by day of month
- same with only 1 box orders

In [ ]:
sns.boxplot(data=exp1, y='pick_seconds', x='day')
plt.title('pick seconds by day of month')
plt.ylim(0, 2000)
plt.show()
# note: y axis is limited to 2000 seconds, or about 30 min to reduce visual impact of outliers

In [ ]:
sns.boxplot(data=X_train_exp, y='pick_seconds', x='day')
plt.title('pick seconds by day of month only orders with 1 box')
plt.ylim(0, 2000)
plt.show()
# note: y axis is limited to 2000 seconds, or about 30 min to reduce visual impact of outliers

### does do the pick seconds vary by week of year?
- some variation possibly due to complexity of order variation?
- not enough of a consistent patter to use as model feature
- minimal change with only 1 box orders

In [ ]:
sns.boxplot(data=exp1, y='pick_seconds', x='week')
plt.title('pick seconds by week of year')
plt.ylim(0, 2000)
plt.show()
# note: y axis is limited to 2000 seconds, or about 30 min to reduce visual impact of outliers

In [ ]:
sns.boxplot(data=X_train_exp, y='pick_seconds', x='week')
plt.title('pick seconds by week of year only orders with 1 box')
plt.ylim(0, 2000)
plt.show()
# note: y axis is limited to 2000 seconds, or about 30 min to reduce visual impact of outliers

### does the pick seconds vary by month of year?
- no variation by month
- no change with only 1 box orders

In [ ]:
sns.boxplot(data=exp1, y='pick_seconds', x='month')
plt.title('pick seconds by month of year')
plt.ylim(0, 2000)
plt.show()
# note: y axis is limited to 2000 seconds, or about 30 min to reduce visual impact of outliers

In [ ]:
sns.boxplot(data=X_train_exp, y='pick_seconds', x='month')
plt.title('pick seconds by month of year only orders with 1 box')
plt.ylim(0, 2000)
plt.show()
# note: y axis is limited to 2000 seconds, or about 30 min to reduce visual impact of outliers

### does the pick seconds vary by year?
- not really any variation by year

In [ ]:
sns.boxplot(data=exp1, y='pick_seconds', x='year')
plt.title('pick seconds by year')
plt.ylim(0, 2000)
plt.show()
# note: y axis is limited to 2000 seconds, or about 30 min to reduce visual impact of outliers

In [ ]:
sns.boxplot(data=X_train_exp, y='pick_seconds', x='year')
plt.title('pick seconds by year only orders with 1 box')
plt.ylim(0, 2000)
plt.show()
# note: y axis is limited to 2000 seconds, or about 30 min to reduce visual impact of outliers

## Explore variation in total lines

This impacts the complexity of the orders

### does the total lines vary by hour of day?
- operating hours start at 7a and end at 6p
- lines are lightest at beginning and end of day with a spike around 5p
- note the y axis has been limited to 50 lines to reduce the visual impact of outliers

In [ ]:
sns.boxplot(data=exp1, y='total_lines', x='hour')
plt.ylim(0, 50)
plt.show()
# note: y axis is limited to 50 lines reduce visual impact of outliers

### does the total lines vary by day of week?
- not much on average, but there seem to be a lot of outliers?

In [ ]:
order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
sns.boxplot(data=exp1, y='total_lines', x='day_name', order=order)
plt.ylim(0, 50)
plt.show()
# note: y axis is limited to 50 lines reduce visual impact of outliers

### does the total lines vary by day of month?
- some variation on average by day of month
- first 4 days of month tend to be higher
- final days of month tend to be lower

In [ ]:
sns.boxplot(data=exp1, y='total_lines', x='day')
plt.ylim(0, 50)
plt.show()
# note: y axis is limited to 50 lines reduce visual impact of outliers

### does the total lines vary by week of year?
- Christmas and Thanksgiving weeks are lighter
- weeks at beginning of months are generally larger and weeks at end of month fewer

In [ ]:
sns.boxplot(data=exp1, y='total_lines', x='week')
plt.ylim(0, 50)
plt.show()
# note: y axis is limited to 50 lines reduce visual impact of outliers

### does the total lines vary by month of year?
- not really any variation by month

In [ ]:
sns.boxplot(data=exp1, y='total_lines', x='month')
plt.ylim(0, 50)
plt.show()
# note: y axis is limited to 50 lines reduce visual impact of outliers

### does the total lines vary by year?
- not really any variation by year

In [ ]:
sns.boxplot(data=exp1, y='total_lines', x='year')

### does the total boxes vary by day of week?
- only outliers?
- typically the boxes per pick is 1 so this may not be a good feature for analysis
- might need to treat values <1 as anomalies and remove from dataset?
    - yes, picks with 0 boxes returned 7 observations that are anomalies and will be removed for this round
- visualization also shows total_boxes over 20 as anomalies
- note y axis is limited to 10 boxes

In [ ]:
order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
sns.boxplot(data=exp1, y='total_boxes', x='day_name', order=order)
plt.ylim(0, 10)
plt.show()
# note: y axis is limited to 10 boxes reduce visual impact of outliers

# Model


- prep for model??
    - drop observations where there is more than 1 box to reduce noise
    - these represent 2% of train dataset
    - YES will drop then create X, y sets then make an X scaled and keep not scaled as well
    
- need to create feature booleen columns
    - only 1 found in data = is hour 18
    
- initial model features total_lines, sec_per_line (scale both of these), and is_hour_18 (boolean)
    - all other columns dropped
    
    
Decided to use Median Absolute Error to evaluate.     
"The median_absolute_error is particularly interesting because it is robust to outliers. The loss is calculated by taking the median of all absolute differences between the target and the prediction."
https://scikit-learn.org/stable/modules/model_evaluation.html#median-absolute-error

## Create Baseline

- Baseline is the mean pick_seconds per order


In [ ]:
baseline = round(y_train.pick_seconds.mean(), 2)
print(f'The average pick seconds per order is', baseline)

In [ ]:
# Predict bps_pred_mean (baseline predicted mean)
bps_pred_mean = y_train['pick_seconds'].mean()
y_train['bps_pred_mean'] = bps_pred_mean
y_validate['bps_pred_mean'] = bps_pred_mean
y_test['bps_pred_mean'] = bps_pred_mean

In [ ]:
# Median Absolute Error (MAE) recommended for non-normally distributed data
# may not matter giving the law of large numbers?

base_med_abs_train = median_absolute_error(y_train.pick_seconds, y_train.bps_pred_mean)
print("MAE using baseline Mean\nTrain/In-Sample: ", round(base_med_abs_train, 2)) 

In [ ]:
evsb = explained_variance_score(y_train.pick_seconds, y_train.bps_pred_mean)
print('Explained Variance = ', round(evsb,3))

## Preprocessing

- create and scale features
- drop unnecessary columns


In [ ]:
X_train.head()

In [ ]:
X_train_scaled, X_validate_scaled, X_test_scaled = wrangle_pick.model_preprocess1(X_train, X_validate, X_test)
X_train_scaled.shape, X_validate_scaled.shape, X_test_scaled.shape

In [ ]:
X_train_scaled.head()

## 1st Round Models

Features = total lines, picked in 18th hour of day

### Linear Regression (OLS) Model

- Ordinary Least Squares

In [ ]:
# create the model object
lm = LinearRegression(normalize=True)
# no change in result if normalize is True or False

# fit the model to our training data. We must specify the column in y_train, 
# since we have converted it to a dataframe from a series! 
lm.fit(X_train_scaled, y_train.pick_seconds)

# predict train
y_train['ps_pred_lm'] = lm.predict(X_train_scaled)

In [ ]:
lm_med_abs_train = median_absolute_error(y_train.pick_seconds, y_train.ps_pred_lm)
print("MAE using Mean\nTrain/In-Sample: ", round(lm_med_abs_train, 2))

### Tweedie Regressor (GLM) Model

- Generalized Linear Model

In [ ]:
# create the model object
glm = TweedieRegressor(power=0, alpha=0)
# tried power (1, 2, 3) and alpha .1 and 1; 0 and 0 provide best result

# fit the model to our training data. We must specify the column in y_train, 
# since we have converted it to a dataframe from a series! 
glm.fit(X_train_scaled, y_train.pick_seconds)

# predict train
y_train['ps_pred_glm'] = glm.predict(X_train_scaled)


In [ ]:
y_train.ps_pred_glm.mean()

In [ ]:
glm_med_abs_train = median_absolute_error(y_train.pick_seconds, y_train.ps_pred_glm)
print("MAE using Mean\nTrain/In-Sample: ", round(glm_med_abs_train, 2))

### Polynomial Feature Model



In [ ]:
# make the polynomial features to get a new set of features
pf = PolynomialFeatures(degree=2)
# fit and transform X_train_scaled
X_train_degree2 = pf.fit_transform(X_train_scaled)
# tried 3, 4, 8 degrees but not much different in performance and 2 is less likely to overfit
    
# transform X_validate_scaled & X_test_scaled
X_validate_degree2 = pf.transform(X_validate_scaled)
X_test_degree2 = pf.transform(X_test_scaled)

# create the model object
lm2 = LinearRegression(normalize=True)

# fit the model to our training data. We must specify the column in y_train, 
# since we have converted it to a dataframe from a series! 
lm2.fit(X_train_degree2, y_train.pick_seconds)

# predict train
y_train['ps_pred_pflm2'] = lm2.predict(X_train_degree2)

In [ ]:
pflm2_med_abs_train = median_absolute_error(y_train.pick_seconds, y_train.ps_pred_pflm2)
print("MAE using Mean\nTrain/In-Sample: ", round(pflm2_med_abs_train, 2))

### Validate

Given that the OLS and GLM model performance is identical I will run the OLS and Polynomial Feature models on the validate dataset.

In [ ]:
# Linear Regreesion predict validate
y_validate['ps_pred_lm'] = lm.predict(X_validate_scaled)

In [ ]:
lm_med_abs_val = median_absolute_error(y_validate.pick_seconds, y_validate.ps_pred_lm)
print("MAE using Mean\nTrain/Out-of-Sample: ", round(lm_med_abs_val, 2))

In [ ]:
# Polynomial Features predict validate
# predict validate
y_validate['ps_pred_pflm2'] = lm2.predict(X_validate_degree2)

In [ ]:
pflm2_med_abs_val = median_absolute_error(y_validate.pick_seconds, y_validate.ps_pred_pflm2)
print("MAE using Mean\nTrain/Out-of-Sample: ", round(pflm2_med_abs_val, 2))

### Test

I will run the Polynomial Feature models on the test dataset.

In [ ]:
# Polynomial Features predict validate
# predict test
y_test['ps_pred_pflm2'] = lm2.predict(X_test_degree2)

In [ ]:
pflm2_med_abs_test = median_absolute_error(y_test.pick_seconds, y_test.ps_pred_pflm2)
print("MAE using Mean\nTrain/Out-of-Sample: ", round(pflm2_med_abs_test, 2))

In [ ]:
evsr1 = explained_variance_score(y_validate.pick_seconds, y_validate.ps_pred_pflm2)
print('Explained Variance = ', round(evsr1,3))

## 1st Round Follow up Models

Is the result the same with only 1 feature?      

Will use OLS and Polynomial models to test this

In [ ]:
# copy X_train_scaled and drop column is_hr_18
X_train_scaled2 = X_train_scaled.copy()
X_train_scaled2 = X_train_scaled2.drop(columns='is_hr_18')
X_validate_scaled2 = X_validate_scaled.copy()
X_validate_scaled2 = X_validate_scaled2.drop(columns='is_hr_18')
X_test_scaled2 = X_test_scaled.copy()
X_test_scaled2 = X_test_scaled2.drop(columns='is_hr_18')
X_train_scaled2.head()

### Linear Regression (OLS) Model

In [ ]:
# create the model object
lm2 = LinearRegression(normalize=True)
# no change in result if normalize is True or False

# fit the model to our training data. We must specify the column in y_train, 
# since we have converted it to a dataframe from a series! 
lm2.fit(X_train_scaled2, y_train.pick_seconds)

# predict train
y_train['ps_pred_lm2'] = lm2.predict(X_train_scaled2)

In [ ]:
lm_med_abs_train2 = median_absolute_error(y_train.pick_seconds, y_train.ps_pred_lm2)
print("MAE using Mean\nTrain/In-Sample: ", round(lm_med_abs_train2, 2))

### Polynomial Feature Model

In [ ]:
# make the polynomial features to get a new set of features
pf2 = PolynomialFeatures(degree=2)
# fit and transform X_train_scaled
X_train_degree22 = pf2.fit_transform(X_train_scaled2)
# tried 3, 4, 8 degrees but not much different in performance and 2 is less likely to overfit
    
# transform X_validate_scaled & X_test_scaled
X_validate_degree22 = pf2.transform(X_validate_scaled2)
X_test_degree22 = pf2.transform(X_test_scaled2)

# create the model object
lm22 = LinearRegression(normalize=True)

# fit the model to our training data. We must specify the column in y_train, 
# since we have converted it to a dataframe from a series! 
lm22.fit(X_train_degree22, y_train.pick_seconds)

# predict train
y_train['ps_pred_pflm22'] = lm22.predict(X_train_degree22)

In [ ]:
pflm22_med_abs_train2 = median_absolute_error(y_train.pick_seconds, y_train.ps_pred_pflm22)
print("MAE using Mean\nTrain/In-Sample: ", round(pflm22_med_abs_train2, 2))

### Validate


In [ ]:
# Linear Regreesion predict validate
y_validate['ps_pred_lm2'] = lm2.predict(X_validate_scaled2)
lm_med_abs_val2 = median_absolute_error(y_validate.pick_seconds, y_validate.ps_pred_lm2)
print("MAE using Mean\nTrain/Out-of-Sample: ", round(lm_med_abs_val2, 2))

In [ ]:
# Polynomial Features predict validate
# predict validate
y_validate['ps_pred_pflm22'] = lm22.predict(X_validate_degree22)
pflm22_med_abs_val2 = median_absolute_error(y_validate.pick_seconds, y_validate.ps_pred_pflm22)
print("MAE using Mean\nTrain/Out-of-Sample: ", round(pflm22_med_abs_val2, 2))

### Test

In [ ]:
# Polynomial Features predict validate
# predict test
y_test['ps_pred_pflm22'] = lm22.predict(X_test_degree22)
pflm22_med_abs_test2 = median_absolute_error(y_test.pick_seconds, y_test.ps_pred_pflm22)
print("MAE using Mean\nTrain/Out-of-Sample: ", round(pflm22_med_abs_test2, 2))

In [ ]:
# sklearn.metrics.explained_variance_score

evsr1f = explained_variance_score(y_test.pick_seconds, y_test.ps_pred_pflm22)
print('Explained Variance = ', round(evsr1f,3))

## 1st Round Results Summary

The MAE (median absolute error) of the baseline (using just the mean pick seconds to predict) is 95 seconds     

Using 2 features only: total lines per order (a measure of order complexity) and if the order is picked in the last hour of the day:
- the 2 degree polynomial model MAE is 33 seconds on both in sample and out of sample data
- this is a 65% improvement in prediction accuracy using the model instead of the mean to predict pick time

Using only 1 feature: total lines produced nearly identical results (with .01 of previous results) I conclude that the is_hr_18 feature is not needed for modeling.      

Prediction accuracy of pick seconds per order can be improved by 65% just using the total lines on the order as a feature in a 2 degree polynomial feature model.

# Explore by Operator


In [ ]:
train.head()

In [ ]:
train.nunique()

In [ ]:
# create aggregate df by operator
operdf = train.groupby(['operator'])['total_lines', 'total_boxes',  'pick_seconds', 'start', 'end', 'hour'].\
                        agg({'total_lines': ['sum'], 'total_boxes' : ['sum', 'count'], 'pick_seconds': ['sum'], \
                             'start': ['min'], 'end': ['max'], 'hour': ['min', 'max']})

# use this to unstack the column names
operdf.columns = [' '.join(col).strip() for col in operdf.columns.values]

# use this for multiple aggregation
# # df.agg({'A' : ['sum', 'min'], 'B' : ['min', 'max']})

#web_crossover.groupby(['name', 'lesson'])[['sub_lesson']].count()

In [ ]:
operdf

In [ ]:
operdf = operdf.rename(columns={'total_lines sum': 'total_lines', 'total_boxes sum': 'total_boxes', \
                                'total_boxes count': 'total_orders', 'pick_seconds sum': 'total_pick_sec', \
                                'start min': 'first_day', 'end max': 'last_day', 'hour min': 'shift_start_hour', \
                                'hour max': 'shift_end_hour'
                                })

In [ ]:
operdf['tenure_days'] = operdf.last_day - operdf.first_day
operdf['shift_length'] = operdf.shift_end_hour - operdf.shift_start_hour
operdf.tenure_days = operdf.tenure_days.dt.days
#operdf.tenure_days = operdf.tenure_days.replace(0, 1)

In [ ]:
# bin by tenure and visualize?
operdf['avg_sec_per_order'] = operdf.total_pick_sec/operdf.total_orders
operdf['avg_line_per_order'] = operdf.total_lines/operdf.total_orders
operdf['avg_box_per_order'] = operdf.total_boxes/operdf.total_orders
operdf['avg_orders_day'] = operdf.total_orders/operdf.tenure_days
operdf

In [ ]:
# can't have less than 1 day of tenure really
operdf.tenure_days == 0

In [ ]:
# create a copy of the dataframe in case this doesn't work
operdf2 = operdf.copy()

In [ ]:
# find where the tenure days is 0 and drop those observations
index = operdf2[operdf2.tenure_days == 0].index
operdf2.drop(index, inplace=True)

In [ ]:
# create bins based on tenure, anyone not in a bin assign to under 90 days bin
operdf2['tenure_bin'] = pd.cut(operdf2.tenure_days, bins=[0, 90, 365, 730, 1095], labels=[.25, 1, 2, 3])
operdf2.tenure_bin = operdf2.tenure_bin.fillna(.25)
operdf2

In [ ]:
# write operdf2 to csv for Tableau
operdf2.to_csv('operator.csv')

## How does the average pick seconds per order vary by operator tenure?

In [ ]:
plt.figure(figsize=(16,7))
sns.boxplot(data=operdf2, y='avg_sec_per_order', x='tenure_bin')
plt.title('Average seconds per order by Tenure bin')
plt.xlabel('Tenure in years')
plt.show()

## How does the order complexity (lines per order) vary by operator tenure?

In [ ]:
plt.figure(figsize=(16,7))
sns.boxplot(data=operdf2, y='avg_line_per_order', x='tenure_bin')
plt.title('Average lines per order by Tenure bin')
plt.xlabel('Tenure in years')
plt.show()

## How does average orders per day vary by tenure?

In [ ]:
plt.figure(figsize=(16,7))
sns.boxplot(data=operdf2, y='avg_orders_day', x='tenure_bin')
plt.title('Average orders per day by Tenure bin')
plt.xlabel('Tenure in years')
plt.show()

## How does the average boxes per order vary by operator tenure?

- note: this is removed for modeling, as only orders with 1 box are being used in model currently

In [ ]:
plt.figure(figsize=(16,7))
sns.boxplot(data=operdf2, y='avg_box_per_order', x='tenure_bin')
plt.title('Average boxes per order by Tenure bin')
plt.xlabel('Tenure in years')
plt.show()

In [ ]:
# create is_PartTime feature
operdf2['is_part_time'] = np.where(operdf2.shift_length <= 6, 1, 0)
operdf2.head()

## How does the average pick seconds vary by part time vs full time operator?

In [ ]:
plt.figure(figsize=(16,7))
sns.boxplot(data=operdf2, y='avg_sec_per_order', x='is_part_time')
plt.title('Average pick seconds per order by shift length, FT=0, PT=1')
plt.show()

### What is the % of FT/PT by tenure bin?

In [ ]:
shift_tenure = pd.crosstab(operdf2.is_part_time, operdf2.tenure_bin, normalize=True, margins=True)
shift_tenure

## How does average orders per day vary by shift length?

In [ ]:
plt.figure(figsize=(16,7))
sns.boxplot(data=operdf2, y='avg_orders_day', x='is_part_time')
plt.title('Average orders per day by shift length, FT=0, PT=1')
plt.show()

## Visualize average pick time by individual operator

In [ ]:
plt.figure(figsize=(16,7))
sns.scatterplot(data=operdf2, y='avg_sec_per_order', x='operator', hue='tenure_bin')
plt.title('Average seconds per order by operator, color designates tenure')
plt.xticks(rotation=90)
plt.show()

In [ ]:
plt.figure(figsize=(16,7))
sns.scatterplot(data=operdf2, y='avg_sec_per_order', x='operator', hue='is_part_time')
plt.title('Average seconds per order by operator, color designates FT=blue/PT=orange')
plt.xticks(rotation=90)
plt.show()

In [ ]:
plt.figure(figsize=(16,7))
sns.scatterplot(data=operdf2, y='avg_sec_per_order', x='operator', hue='avg_line_per_order')
plt.title('Average seconds per order by operator, color designates avg_line_per_order')
plt.xticks(rotation=90)
plt.show()

# 2nd round Modeling

## 2nd round preprocessing

- want to add features based on the operator tenure and part time status
- creating copy of X_train so don't overwrite current dataframe


In [ ]:
train.head()

In [ ]:
operdf2.head()

In [ ]:
# create column in X train that looks up tenure bin value in operdf2 for the operator list
# create column in X train that looks up is_part_time status of operator in operdf2

# not sure what will happen if merge all columns, but only want tenure bin and is part time
# create copy of operdf2 that has only operator tenure bin and is part time
# https://www.geeksforgeeks.org/how-to-do-a-vlookup-in-python-using-pandas/

oper_mergedf = operdf2[['tenure_bin', 'is_part_time']]
oper_mergedf = oper_mergedf.reset_index()

In [ ]:
oper_mergedf.head()

In [ ]:
newtraindf = pd.merge(train, oper_mergedf, on='operator', how='left')
newtraindf
# this will need to be preprocessed before modeling

In [ ]:
# create validate and test with new features
newvaldf = pd.merge(validate, oper_mergedf, on='operator', how='left') 
newtestdf = pd.merge(test, oper_mergedf, on='operator', how='left')

In [ ]:
newtraindf[newtraindf.tenure_bin.isna()]

In [ ]:
newvaldf.isna().sum()

In [ ]:
newtestdf.isna().sum()

In [ ]:
# after merge there are 236 out of 96K rows with null values in train
# validate and test have 52 and 53 rows with null values
# dropping these observations for modeling
newtraindf = newtraindf.dropna()
newvaldf = newvaldf.dropna()
newtestdf = newtestdf.dropna()

In [ ]:
# recheck for nulls
newtraindf.isna().sum()

In [ ]:
# need to do X y split on new dataframes
newX_train_exp, newX_train, newX_validate, newX_test, newy_train, newy_validate, newy_test = wrangle_pick.createXy(newtraindf, newvaldf, newtestdf)

In [ ]:
# preprocess round 2 train, validate, test
r2X_train, r2X_validate, r2X_test = wrangle_pick.model_preprocess2(newX_train, newX_validate, newX_test)

In [ ]:
r2X_train.shape, r2X_validate.shape, r2X_test.shape

## 2nd Round Models

Will use OSL and polynomial features (2 degrees) as before and continue using MAE (Median Absolute Error) for evaluation of the models.

First evaluate using only tenure bin and is part time, follow up with only tenure bin

In [ ]:
r2_train_oper_features = r2X_train.drop(columns='total_lines_scaled')
r2_val_oper_features = r2X_validate.drop(columns='total_lines_scaled')
r2_test_oper_features = r2X_test.drop(columns='total_lines_scaled')
r2_train_oper_features.head()

### Linear Regression (OLS) Model

In [ ]:
# create the model object
r2lm = LinearRegression(normalize=True)
# no change in result if normalize is True or False

# fit the model to our training data. We must specify the column in y_train, 
# since we have converted it to a dataframe from a series! 
r2lm.fit(r2_train_oper_features, newy_train.pick_seconds)

# predict train
newy_train['ps_pred_r2lm'] = r2lm.predict(r2_train_oper_features)

In [ ]:
r2lm_med_abs_train = median_absolute_error(newy_train.pick_seconds, newy_train.ps_pred_r2lm)
print("MAE using Mean\nTrain/In-Sample: ", round(r2lm_med_abs_train, 2))

In [ ]:
evsr2 = explained_variance_score(newy_train.pick_seconds, newy_train.ps_pred_r2lm)
print('Explained Variance = ', round(evsr2,3))

### Polynomial Feature Model

In [ ]:
# make the polynomial features to get a new set of features
r2pf = PolynomialFeatures(degree=2)
# fit and transform X_train_scaled
r2X_train_degree = r2pf.fit_transform(r2_train_oper_features)
# tried 3, 4, 8 degrees but not much different in performance and 2 is less likely to overfit
    
# transform X_validate_scaled & X_test_scaled
r2X_validate_degree = r2pf.transform(r2_val_oper_features)
r2X_test_degree = r2pf.transform(r2_test_oper_features)

# create the model object
r2lmpf = LinearRegression(normalize=True)

# fit the model to our training data. We must specify the column in y_train, 
# since we have converted it to a dataframe from a series! 
r2lmpf.fit(r2X_train_degree, newy_train.pick_seconds)

# predict train
newy_train['ps_pred_r2pflm'] = r2lmpf.predict(r2X_train_degree)

In [ ]:
r2pflm_med_abs_train = median_absolute_error(newy_train.pick_seconds, newy_train.ps_pred_r2pflm)
print("MAE using Mean\nTrain/In-Sample: ", round(r2pflm_med_abs_train, 2))

## 2nd Round Results Summary
using only operator features is about equal to baseline

No need for further validate or test due to poor performance

## 3rd Round Models

For this final set will use total line with operator features

In [ ]:
r2X_train.shape, r2X_validate.shape, r2X_test.shape

### Linear Regression (OLS) Model

In [ ]:
# create the model object
r3lm = LinearRegression(normalize=True)
# no change in result if normalize is True or False

# fit the model to our training data. We must specify the column in y_train, 
# since we have converted it to a dataframe from a series! 
r3lm.fit(r2X_train, newy_train.pick_seconds)

# predict train
newy_train['ps_pred_r3lm'] = r3lm.predict(r2X_train)

In [ ]:
r3lm_med_abs_train = median_absolute_error(newy_train.pick_seconds, newy_train.ps_pred_r3lm)
print("MAE using Mean\nTrain/In-Sample: ", round(r3lm_med_abs_train, 2))

### Polynomial Feature Model

In [ ]:
# make the polynomial features to get a new set of features
r3pf = PolynomialFeatures(degree=2)
# fit and transform X_train_scaled
r3X_train_degree = r3pf.fit_transform(r2X_train)
# tried 3, 4, 8 degrees but not much different in performance and 2 is less likely to overfit
    
# transform X_validate_scaled & X_test_scaled
r3X_validate_degree = r3pf.transform(r2X_validate)
r3X_test_degree = r3pf.transform(r2X_test)

# create the model object
r3lmpf = LinearRegression(normalize=True)

# fit the model to our training data. We must specify the column in y_train, 
# since we have converted it to a dataframe from a series! 
r3lmpf.fit(r3X_train_degree, newy_train.pick_seconds)

# predict train
newy_train['ps_pred_r3pflm'] = r3lmpf.predict(r3X_train_degree)

In [ ]:
r3pflm_med_abs_train = median_absolute_error(newy_train.pick_seconds, newy_train.ps_pred_r3pflm)
print("MAE using Mean\nTrain/In-Sample: ", round(r3pflm_med_abs_train, 2))

### Validate


In [ ]:
# Linear Regreesion predict validate
newy_validate['r3ps_pred_lm'] = r3lm.predict(r2X_validate)
r3lm_med_abs_val = median_absolute_error(newy_validate.pick_seconds, newy_validate.r3ps_pred_lm)
print("MAE using Mean\nTrain/Out-of-Sample: ", round(r3lm_med_abs_val, 2))

In [ ]:
# Polynomial Features predict validate
# predict validate
newy_validate['r3ps_pred_pflm'] = r3lmpf.predict(r3X_validate_degree)
r3pflm_med_abs_val = median_absolute_error(newy_validate.pick_seconds, newy_validate.r3ps_pred_pflm)
print("MAE using Mean\nTrain/Out-of-Sample: ", round(r3pflm_med_abs_val, 2))

### Test

In [ ]:
# Polynomial Features predict validate
# predict test
newy_test['r3ps_pred_pflm'] = r3lmpf.predict(r3X_test_degree)
r3pflm_med_abs_test = median_absolute_error(newy_test.pick_seconds, newy_test.r3ps_pred_pflm)
print("MAE using Mean\nTrain/Out-of-Sample: ", round(r3pflm_med_abs_test, 2))

In [ ]:
evsr3 = explained_variance_score(newy_test.pick_seconds, newy_test.r3ps_pred_pflm)
print('Explained Variance = ', round(evsr3,3))

## 3rd Round Results Summary

The MAE (median absolute error) of the baseline (using just the mean pick seconds to predict) is 95 seconds     

Using 3 features: total lines per order (a measure of order complexity), binned operator tenure, and part time status does not improve performance from using total lines alone.

The minimal variation between performance of operators based on tenure time and/or FT/PT status in this dataset may be a factor. In a dataset with greater variation these factors might have more predictive impact?

In [ ]:
newy_validate['baseline_pred'] = bps_pred_mean
newy_validate.head()

In [ ]:

plt.figure(figsize=(16,8))
plt.plot(newy_validate.pick_seconds, newy_validate.baseline_pred, alpha=.5, color="gray", label='_nolegend_')
plt.annotate("Baseline: Predict Using Mean", (16, 9.5))
plt.plot(newy_validate.pick_seconds, newy_validate.pick_seconds, alpha=.5, color="blue", label='_nolegend_')
plt.annotate("The Ideal Line: Predicted = Actual", (.5, 3.5), rotation=25.5)

plt.scatter(newy_validate.pick_seconds, newy_validate.r3ps_pred_lm, 
            alpha=.5, color="red", s=100, label="Model: LinearRegression")

plt.scatter(newy_validate.pick_seconds, newy_validate.r3ps_pred_pflm, 
            alpha=.5, color="green", s=100, label="Model 2nd degree Polynomial")
plt.legend()
plt.xlabel("Actual pick seconds")
plt.ylabel("Predicted pick seconds")
plt.title("Where are predictions more extreme? More modest?")
# plt.annotate("The polynomial model appears to overreact to noise", (2.0, -10))
# plt.annotate("The OLS model (LinearRegression)\n appears to be most consistent", (15.5, 3))
plt.show()

In [ ]:
plt.figure(figsize=(16,8))
plt.axhline(label="No Error")
plt.scatter(newy_validate.pick_seconds, newy_validate.r3ps_pred_pflm-newy_validate.pick_seconds, 
            alpha=.5, color="red", s=100, label="Model: LinearRegression")

plt.scatter(newy_validate.pick_seconds, newy_validate.r3ps_pred_pflm-newy_validate.pick_seconds, 
            alpha=.5, color="green", s=100, label="Model 2nd degree Polynomial")
plt.legend()
plt.xlabel("Actual pick seconds")
plt.ylabel("Residual/Error: Predicted pick seconds - Actual pick seconds")
plt.title("Do the size of errors change as the actual value changes?")
#plt.annotate("The polynomial model appears to overreact to noise", (2.0, -10))
#plt.annotate("The OLS model (LinearRegression)\n appears to be most consistent", (15.5, 3))
plt.show()

In [ ]:
f_pval = ols_model.f_pvalue

print("p-value for model significance = ", round(f_pval,4))

In [ ]:
# sklearn.metrics.explained_variance_score

evs = explained_variance_score(df.y, df.yhat)
print('Explained Variance = ', round(evs,3))